<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [3]:
!pip install -r ../requirements.txt --user

  Cloning git://github.com/dvdpedros/pyfolio (to revision fix-extreme-drawdown) to /private/var/folders/_4/4l2h0xdn6n1d86jjq29rfqdm0000gp/T/pip-req-build-hbfoek49
  Running command git clone -q git://github.com/dvdpedros/pyfolio /private/var/folders/_4/4l2h0xdn6n1d86jjq29rfqdm0000gp/T/pip-req-build-hbfoek49
  Running command git checkout -b fix-extreme-drawdown --track origin/fix-extreme-drawdown
  Switched to a new branch 'fix-extreme-drawdown'
  Branch 'fix-extreme-drawdown' set up to track remote branch 'fix-extreme-drawdown' from 'origin'.


distutils: /Users/davidpedros/Library/Python/3.8/lib/python/site-packages
sysconfig: /Users/davidpedros/Library/Python/3.8/lib/python3.8/site-packages
distutils: /Users/davidpedros/Library/Python/3.8/lib/python/site-packages
sysconfig: /Users/davidpedros/Library/Python/3.8/lib/python3.8/site-packages
distutils: /Users/davidpedros/Library/Python/3.8/include/python3.8/UNKNOWN
sysconfig: /Users/davidpedros/Library/Python/3.8/include/python3.8
user = True
home = None
root = None
prefix = None
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


<a id='1.2'></a>
## 2.2. Import Packages

In [1]:
%load_ext autoreload
%autoreload 
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from src.config import config
from src.dataset.cryptodownloader import CryptoDownloader
from src.preprocessing.preprocessors import FeatureEngineer
from src.preprocessing.data import data_split
from src.environment.env_custom import CustomTradingEnv
from src.model.models import DRLAgent
from src.evaluate.backtest import BackTest
import warnings
warnings.filterwarnings("ignore")



/usr/local/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.3'></a>
## 2.3 Create Folders

In [2]:
import os
download_data = False
if not os.path.exists(config.DATA_SAVE_DIR):
    os.makedirs(config.DATA_SAVE_DIR)
    download_data = True
if not os.path.exists(config.TRAINED_MODEL_DIR):
    os.makedirs(config.TRAINED_MODEL_DIR)
if not os.path.exists(config.TENSORBOARD_LOG_DIR):
    os.makedirs(config.TENSORBOARD_LOG_DIR)
if not os.path.exists(config.RESULTS_DIR):
    os.makedirs(config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [3]:
data_downloader = CryptoDownloader(config.START_DATE, config.END_DATE, config.MULTIPLE_TICKER_8, config.DATA_SAVE_DIR, config.DATA_GRANULARITY)
if download_data:    
    data_downloader.download_data()
df = data_downloader.load()

In [4]:
df

,date,open,high,low,close,volume,tic
0,2020-01-01 00:00:00,7195.24,7196.25,7175.46,7177.02,511.814901,btc
1,2020-01-01 01:00:00,7176.47,7230.00,7175.71,7216.27,883.052603,btc
2,2020-01-01 02:00:00,7215.52,7244.87,7211.41,7242.85,655.156809,btc
3,2020-01-01 03:00:00,7242.66,7245.00,7220.00,7225.01,783.724867,btc
4,2020-01-01 04:00:00,7225.00,7230.00,7215.03,7217.27,467.812578,btc
...,...,...,...,...,...,...,...
105019,2021-06-30 19:00:00,140.83,141.76,140.33,141.55,21511.253020,ltc
105020,2021-06-30 20:00:00,141.63,142.91,140.70,142.16,32348.880040,ltc
105021,2021-06-30 21:00:00,142.16,143.86,142.14,143.25,23128.015840,ltc
105022,2021-06-30 22:00:00,143.25,144.41,142.39,143.50,22183.430460,ltc


# Part 4: Preprocess Data

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = True,
                    use_covariance= True
)

df = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added user defined features


In [ ]:
df.head()

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2020-02-02 to 2020-12-31

In [ ]:
train = data_split(df, '2020-02-02','2020-12-31')

## Environment for Portfolio Allocation


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
features =  config.TECHNICAL_INDICATORS_SHORTPERIOD + config.TECHNICAL_INDICATORS_LONGPERIOD  + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]
features += [feature for feature in df.columns if feature.startswith("cov_")]

env_kwargs = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": train.tic.unique(),
    "all_tickers": train.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}

e_train_gym = CustomTradingEnv(df = train, **env_kwargs)

env_train, _ = e_train_gym.get_sb_env()

<a id='5'></a>
# Part 6: Implement DRL Algorithms

In [ ]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [ ]:
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS, tensorboard_log = config.TENSORBOARD_LOG_DIR)

In [ ]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=100)

### Model 2: **PPO**

In [ ]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

## Trading
Assume that we have $1,000,000 initial capital at 2021-01-01.

In [ ]:
trade = data_split(df,config.START_TEST_DATE, config.END_DATE)
e_trade_gym = CustomTradingEnv(df = trade, **env_kwargs)

In [ ]:
trade.shape

In [ ]:
trade

In [ ]:
allocations, transactions, allocation_values = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

In [ ]:
allocations

In [ ]:
transactions

In [ ]:
allocation_values

<a id='6'></a>
# Part 7: Backtest

In [ ]:
%load_ext autoreload
%autoreload 
bat = BackTest(trained_a2c, e_trade_gym)
bat.create_summary(allocations, allocation_values, transactions)

In [ ]:
bat.plot_return_against_hold(allocation_values)

In [ ]:
%load_ext autoreload
%autoreload 
from src.trade.time_series_validation import TimeSeriesValidation
tsv = TimeSeriesValidation(num_splits=5, total_timesteps_model=1000, with_graphs=False)

model_name = "a2c"
model_params = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}

stock_dimension = len(df.tic.unique())
state_space = stock_dimension
features =  config.TECHNICAL_INDICATORS_SHORTPERIOD + config.TECHNICAL_INDICATORS_LONGPERIOD + ["open", "close", 'high', 'low']
features += [f"{feature}_diff" for feature in features]
features += [feature for feature in df.columns if feature.startswith("cov_")]

env_params = {
    "initial_amount": 1000000, 
    "technical_indicator_list": features, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": df.tic.unique(),
    "all_tickers": df.tic.unique(),
    "reward_scaling": 1e-4,
    "comission_value": 0.01
    
}


results = tsv.run(df, env_params, model_name, model_params)

In [ ]:
results